In [4]:
from PIL import Image
from TCFile import TCFile
import utils
import os

# configurations
times = [36]
mode = 'mip'
crop_size = (480, 480)
save_path = f'C:/rkka_Projects/cell_death_v2/Data/model_training_HDF/{mode}/data'
root_dir = r"C:\rkka_Projects\cell_death_v2\Data\1_HDF"

min, max = 1.33, 1.40

file_list = os.listdir(r"C:\rkka_Projects\cell_death_v2\Data\1_HDF")

data = {}
# Crop
for file_path in file_list:
    data[file_path] = []
    file = TCFile(os.path.join(root_dir, file_path), '3D')
    for time_index in times:
        slice_2d = utils.resize_tomogram.resize_tomogram_2d(file[time_index], data_resolution=file.data_resolution,
                                                target_resolution=0.1632, mode='mip')
            
        patches = utils.crop_patch(slice_2d, crop_size=crop_size)
        for patch_index, patch in enumerate(patches):
            top = patch[0]
            left = patch[1]
            temp = slice_2d[top:top+crop_size[0], left:left+crop_size[1]]
            temp = utils.image_normalization(temp, min=min, max=max)
            image = Image.fromarray(temp)
            data[file_path].append(image)
print('finished!')

c:\miniconda3\envs\cell\Lib\site-packages\TCFile\TCFile_class.py:190: UserWarning: You use an experimental file format deprecated.
Update your reconstruction program and rebuild TCF file.
  warnings.warn(("You use an experimental file format deprecated.\n"


finished!


In [5]:
data

{'230728.165757.HDF_CD95.001.Group1.A1.T001P01.TCF': [<PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>],
 '230728.165757.HDF_CD95.001.Group1.A1.T001P02.TCF': [<PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>],
 '230728.165757.HDF_CD95.001.Group1.A1.T001P03.TCF': [<PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>],
 '230728.165757.HDF_CD95.001.Group1.A1.T001P04.TCF': [<PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>,
  <PIL.Image.Image image mode=L size=480x480>],
 '230728.165757.HDF_CD95.001.Group1.

In [6]:
import utils
from torchvision import models
import torch

model = models.resnet101(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.2),
    torch.nn.Linear(num_features, 4)
)
model.load_state_dict(torch.load('test.pth'))

c:\miniconda3\envs\cell\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\miniconda3\envs\cell\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\김민욱\AppData\Local\Temp\ipykernel_10132\2823146596.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURIT

<All keys matched successfully>

In [26]:
temp = data['230728.165757.HDF_CD95.001.Group1.A1.T001P02.TCF']
temp

[<PIL.Image.Image image mode=L size=480x480>,
 <PIL.Image.Image image mode=L size=480x480>,
 <PIL.Image.Image image mode=L size=480x480>,
 <PIL.Image.Image image mode=L size=480x480>]

In [27]:
import numpy as np

transform = models.ResNet101_Weights.IMAGENET1K_V2.transforms()

results = torch.zeros((1,4))

model.eval()
with torch.no_grad():
    for t in temp:
        image = t
        image = np.array(image)
        image_tensor = torch.from_numpy(image).repeat(3,1,1).float()
        image_tensor = transform(image_tensor)
        
        output = model(image_tensor.unsqueeze(0))
        results += output[0]
    
    results = results / 4

In [28]:
results

tensor([[-0.2955,  2.0160, -1.2532, -0.3927]])

In [23]:
import napari
viewer = napari.Viewer()

In [24]:
path = r"C:\rkka_Projects\cell_death_v2\Data\1_HDF\230728.165757.HDF_CD95.001.Group1.A1.T001P01.TCF"
file = TCFile(path, '2DMIP')

In [25]:
viewer.add_image(np.array([s for s in file]))

<Image layer 'Image' at 0x29cc41c7890>